<a href="https://colab.research.google.com/github/Annie00000/Project/blob/main/6_13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. 指定路徑

In [ ]:
import glob
import os

folder_path = '/your/path/here'
prefix = 'chart_aaa'  # 動態變數

# 方法 1: glob :  組合 pattern，例如 chart_aaa__*.png
pattern = os.path.join(folder_path, f'{prefix}__*.png')
png_files = glob.glob(pattern)


# 方法 2:
import os
folder_path = '/your/path/here'
png_files = [
    f for f in os.listdir(folder_path)
    if f.startswith('chart_aaa') and f.endswith('.png')
]
# 若需要完整路徑
png_files = [os.path.join(folder_path, f) for f in png_files]

## 2. 放照片

In [ ]:
### key 對應 base64路徑
import base64
import os

for file in png_files:
    file_key = file[:-4].split('___')[1]  # 去掉最後4個字元 '.png'
    full_path = os.path.join(folder_path, file)
    with open(full_path, 'rb') as f:
        encoded = base64.b64encode(f.read()).decode()
    image_map[file_key] = f'data:image/png;base64,{encoded}'




#### 2-1. html.Div()

In [ ]:
import os
import dash
from dash import html

app = dash.Dash(__name__)

# 圖片所在資料夾與前綴
folder_path = '../../../data_source/stat/tm'
prefix = 'chart_aaa___'

# 搜尋符合條件的圖片檔案
all_files = os.listdir(folder_path)
png_files = [f for f in all_files if f.startswith(prefix) and f.endswith('.png')]

# 根據結尾關鍵字放入對應欄位
image_map = {
    'CPK': None,
    'MEAN_SHIFT': None,
    'SIGMA_RATIO': None,
    'TREND_CHART': None,
}

for file in png_files:
    for key in image_map:
        if file.endswith(f'{key}.png'):
            image_map[key] = os.path.join(folder_path, file)


## 轉base64
"""
### 簡潔 :key 對應 base64路徑
import base64
import os

for file in png_files:
    file_key = file[:-4].split('___')[1]  # 去掉最後4個字元 '.png'
    full_path = os.path.join(folder_path, file)
    with open(full_path, 'rb') as f:
        encoded = base64.b64encode(f.read()).decode()
    image_map[file_key] = f'data:image/png;base64,{encoded}'

"""

"""
import base64
import os

for file in png_files:
    for key in image_map:
        if file.endswith(f'{key}.png'):
            full_path = os.path.join(folder_path, file)
            with open(full_path, 'rb') as f:
                encoded = base64.b64encode(f.read()).decode()
            image_map[key] = f'data:image/png;base64,{encoded}'
            break
"""

# 建立圖片元件
def image_component(path):
    if path and os.path.exists(path):
        # Dash 本身無法直接讀本機路徑，這邊簡化寫法為占位用
        return html.Img(src=path, style={'width': '100%', 'height': 'auto', 'border': '1px solid #ccc'})
    else:
        return html.Div(style={'width': '100%', 'height': '300px'})  # 空白占位

# 頁面 layout：2x2 grid
app.layout = html.Div([
    html.Div([
        # 左上、右上
        html.Div(image_component(image_map['CPK']), style={'width': '50%'}),
        html.Div(image_component(image_map['MEAN_SHIFT']), style={'width': '50%'})
    ], style={'display': 'flex'}),

    html.Div([
        # 左下、右下
        html.Div(image_component(image_map['SIGMA_RATIO']), style={'width': '50%'}),
        html.Div(image_component(image_map['TREND_CHART']), style={'width': '50%'})
    ], style={'display': 'flex'})
])


#### 2-2. dbc.

In [ ]:
import os
import dash
from dash import html
import dash_bootstrap_components as dbc

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# 設定資料夾與檔名前綴
folder_path = '../../../data_source/stat/tm'
prefix = 'chart_aaa___'

# 找出所有符合 prefix 的檔案
all_files = os.listdir(folder_path)
png_files = [f for f in all_files if f.startswith(prefix) and f.endswith('.png')]

# 建立對應字典：{分類名稱: 圖檔路徑}
image_map = {
    'CPK': None,
    'MEAN_SHIFT': None,
    'SIGMA_RATIO': None,
    'TREND_CHART': None,
}

# 依據檔名結尾放入對應位置
for file in png_files:
    for key in image_map:
        if file.endswith(f'{key}.png'):
            image_map[key] = os.path.join(folder_path, file)
            break

# 建立圖片元件（若為 None 則留空）
def image_component(path):
    if path and os.path.exists(path):
        return html.Img(src=path, style={'width': '100%', 'height': 'auto', 'border': '1px solid #ccc'})
    else:
        return html.Div()  # 空白占位

# Layout：2x2 Grid
app.layout = dbc.Container([
    dbc.Row([
        dbc.Col(image_component(image_map['CPK']), width=6),  # 示佔滿整體寬度的 6/12（Bootstrap 網格系統總寬度為 12）
        dbc.Col(image_component(image_map['MEAN_SHIFT']), width=6),
    ]),
    dbc.Row([
        dbc.Col(image_component(image_map['SIGMA_RATIO']), width=6),
        dbc.Col(image_component(image_map['TREND_CHART']), width=6),
    ]),
], fluid=True)

if __name__ == '__main__':
    app.run_server(debug=True)


### 3. PPT

In [ ]:
import os
from pptx import Presentation
from pptx.util import Inches, Pt

# 範例元素清單
elements = ['chartA', 'chartB', 'chartC']

image_folder = './images'

prs = Presentation()

# 4個位置座標 (左上, 右上, 左下, 右下)
positions = {
    'CPK': (Inches(0.5), Inches(1.5)), # 距離左邊 0.5 吋，距離上方 1.5 吋。
    'MEAN_SHIFT': (Inches(5.0), Inches(1.5)),
    'SIGMA_RATIO': (Inches(0.5), Inches(4.5)),
    'TREND_CHART': (Inches(5.0), Inches(4.5)),
}

img_width = Inches(4.5)
img_height = Inches(3)

for elem in elements:
    slide = prs.slides.add_slide(prs.slide_layouts[5])  # 空白版型

    # 加標題
    title_box = slide.shapes.add_textbox(Inches(0.5), Inches(0.2), Inches(9), Inches(1))
    title_frame = title_box.text_frame
    title_frame.text = elem
    title_frame.paragraphs[0].font.size = Pt(28)

    # 找該元素相關png圖片
    # 篩選以 elem 開頭，且結尾含特定字串的png
    relevant_imgs = [f for f in os.listdir(image_folder) if f.startswith(elem) and f.lower().endswith('.png')]

    # 用來記錄每個位置是否有圖
    placed = {key: False for key in positions.keys()}

    for img_file in relevant_imgs:
        for key in positions.keys():
            if img_file.endswith(f"_{key}.png"):
                left, top = positions[key]
                img_path = os.path.join(image_folder, img_file)
                slide.shapes.add_picture(img_path, left, top, width=img_width, height=img_height)
                placed[key] = True
                break  # 找到位置後跳出內層

    # 沒找到該位置的圖片會留空白，不用特別處理

prs.save('output_2x2_layout.pptx')


In [ ]:
### 比對一下跟上面差別

import os
from pptx import Presentation
from pptx.util import Inches, Pt

elements = ['chartA', 'chartB', 'chartC']
image_folder = './images'

prs = Presentation()

positions = {
    'CPK': (Inches(0.5), Inches(1.5)),
    'MEAN_SHIFT': (Inches(5.0), Inches(1.5)),
    'SIGMA_RATIO': (Inches(0.5), Inches(4.5)),
    'TREND_CHART': (Inches(5.0), Inches(4.5)),
}

img_width = Inches(4.5)
img_height = Inches(3)

for elem in elements:
    slide = prs.slides.add_slide(prs.slide_layouts[5])

    title_box = slide.shapes.add_textbox(Inches(0.5), Inches(0.2), Inches(9), Inches(1))
    title_frame = title_box.text_frame
    title_frame.text = elem
    title_frame.paragraphs[0].font.size = Pt(28)

    # 用 dict 記錄每個 key 對應的圖片路徑 (只記第一張)
    img_for_key = {}

    # 掃描所有符合 elem 開頭的圖片
    relevant_imgs = [f for f in os.listdir(image_folder) if f.startswith(elem) and f.lower().endswith('.png')]

    for img_file in relevant_imgs:
        for key in positions.keys():
            if img_file.endswith(f"_{key}.png") and key not in img_for_key:
                img_for_key[key] = os.path.join(image_folder, img_file)

    # 放圖片
    for key, pos in positions.items():
        if key in img_for_key:
            left, top = pos
            slide.shapes.add_picture(img_for_key[key], left, top, width=img_width, height=img_height)
        # 找不到圖片則跳過，該位置留空

prs.save('output_2x2_layout_no_break.pptx')
